In [1]:
%matplotlib inline

In [2]:
path = "../data/"

In [3]:
from __future__ import division,print_function

import os, json
from glob import glob
import numpy as np
import scipy
from sklearn.preprocessing import OneHotEncoder
from sklearn.metrics import confusion_matrix
np.set_printoptions(precision=4, linewidth=100)
from matplotlib import pyplot as plt
import utils; reload(utils)
from utils import plots, get_batches, plot_confusion_matrix, get_data
from numpy.random import random, permutation
from scipy import misc, ndimage
from scipy.ndimage.interpolation import zoom

import keras
from keras import backend as K
from keras.utils.data_utils import get_file
from keras.models import Sequential
from keras.layers import Input
from keras.layers.core import Flatten, Dense, Dropout, Lambda
from keras.layers.convolutional import Convolution2D, MaxPooling2D, ZeroPadding2D
from keras.optimizers import SGD, RMSprop
from keras.preprocessing import image

Using gpu device 0: GeForce GTX 1080 (CNMeM is enabled with initial size: 90.0% of memory, cuDNN 5005)
Using Theano backend.


In [4]:
import vgg16; reload(vgg16)
from vgg16 import Vgg16

In [5]:
vgg = Vgg16()

In [6]:
vgg.model.pop()
for layer in vgg.model.layers: layer.trainable=False

In [7]:
vgg.model.add(Dense(3, activation='softmax'))

In [8]:
# layers = vgg.model.layers
# # Get the index of the first dense layer...
# first_dense_idx = [index for index,layer in enumerate(layers) if type(layer) is Dense][0]
# # ...and set this and all subsequent layers to trainable
# for layer in layers[first_dense_idx:]: layer.trainable=True

In [9]:
trn_data = get_data(path+'train')
val_data = get_data(path+'valid')
# Use batch size of 1 since we're just doing preprocessing on the CPU
val_batches = get_batches(path+'valid', shuffle=False, batch_size=1)
batches = get_batches(path+'train', shuffle=False, batch_size=1)
val_classes = val_batches.classes
trn_classes = batches.classes
def onehot(x): return np.array(OneHotEncoder().fit_transform(x.reshape(-1,1)).todense())
val_labels = onehot(val_classes)
trn_labels = onehot(trn_classes)

Found 1196 images belonging to 3 classes.
Found 284 images belonging to 3 classes.
Found 284 images belonging to 3 classes.
Found 1196 images belonging to 3 classes.


In [10]:
batch_size = 64
gen=image.ImageDataGenerator()

batches = gen.flow(trn_data, trn_labels, batch_size=batch_size, shuffle=True)
val_batches = gen.flow(val_data, val_labels, batch_size=batch_size, shuffle=False)

In [11]:
opt = RMSprop(lr=0.1)
vgg.model.compile(optimizer=opt, loss='categorical_crossentropy', metrics=['accuracy'])

In [12]:
def fit_model(model, batches, val_batches, nb_epoch=1):
    model.fit_generator(batches, samples_per_epoch=batches.n, nb_epoch=nb_epoch, 
                        validation_data=val_batches, nb_val_samples=val_batches.n)
fit_model(vgg.model, batches, val_batches, nb_epoch=2)

Epoch 1/2
1196/1196 [==============================] - 14s - loss: 7.1036 - acc: 0.5343 - val_loss: 8.6266 - val_acc: 0.4648

In [13]:
K.set_value(opt.lr, 0.01)
fit_model(vgg.model, batches, val_batches, 3)

Epoch 1/3
1196/1196 [==============================] - 15s - loss: 7.4796 - acc: 0.5360 - val_loss: 8.6266 - val_acc: 0.4648

In [14]:
path = "../data/"
batches = vgg.get_batches(batch_size = batch_size, path = path + 'train')
val_batches = vgg.get_batches(batch_size = batch_size, path = path + 'valid')
vgg.finetune(batches)

Found 1196 images belonging to 3 classes.
Found 284 images belonging to 3 classes.


In [15]:
vgg.fit(batches, val_batches, nb_epoch=5)

Epoch 1/5
1196/1196 [==============================] - 14s - loss: 1.9510 - acc: 0.4415 - val_loss: 1.2430 - val_acc: 0.3768

In [16]:
vgg.fit(batches, val_batches, nb_epoch=1)

Epoch 1/1
1196/1196 [==============================] - 15s - loss: 1.2749 - acc: 0.5443 - val_loss: 1.0255 - val_acc: 0.5141

In [17]:
b,p = vgg.test(path = path + 'test', batch_size=2)
prediction = zip([name[8:] for name in b.filenames], p.astype('str'))
outF = open('retrain_dense_layers.csv', 'w')
outF.write('image_name,Type_1,Type_2,Type_3\n')
for elem in prediction:
    outF.write(elem[0] + ',' + ','.join(elem[1]) + '\n')
outF.close()

Found 512 images belonging to 1 classes.
